In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.metrics import roc_curve
from tpot import TPOTClassifier, TPOTRegressor

/Users/ellenhoeven/anaconda3/envs/wids2021/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


# Import data and very quick preprocessing
- Create target and features
- Drop columns that are not needed
- Encode categorical columns

In [55]:
data = pd.read_csv("TrainingWiDS2021.csv")

In [71]:
data.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,...,NaN,NaN,0,0,0,0,0,0,0,0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,...,337.0,337.0,0,0,0,0,0,0,0,0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0


In [72]:
y = data['diabetes_mellitus']

In [73]:
X = data.drop(['diabetes_mellitus'], axis=1)

In [74]:
X.drop("Unnamed: 0", axis = 1, inplace = True)
X.drop("encounter_id", axis = 1, inplace = True)
X.drop("hospital_id", axis = 1, inplace = True)
X.drop("icu_id", axis = 1, inplace = True)
X.drop("icu_stay_type", axis = 1, inplace = True)

In [75]:
categorical_cols = ['elective_surgery',
                   'ethnicity',
                   'gender',
                   'hospital_admit_source',
                   'icu_admit_source',
                   'icu_type',
                   'readmission_status']

In [76]:
X = pd.get_dummies(X, columns=categorical_cols)

In [77]:
# list(X.dtypes)

In [78]:
# X_tmp = X.select_dtypes(exclude=['uint8'])

In [79]:
y.dtypes

dtype('int64')

In [80]:
y.unique()

array([1, 0])

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(),
                                                    train_size=0.75, test_size=0.25, 
                                                    random_state=42)

In [82]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(97617, 207)
(32540, 207)
(97617,)
(32540,)


# Start TPOT

https://machinelearningmastery.com/tpot-for-automated-machine-learning-in-python/

http://epistasislab.github.io/tpot/using/

### Create a first simple pipeline to test if everything works

In [84]:
tpot = TPOTClassifier(generations=2, 
                      population_size=2, 
                      verbosity=2, random_state=42)

In [85]:
# pipeline_optimizer.fit(X_train, y_train)
tpot.fit(X_train, y_train)

Version 0.11.6.post3 of tpot is outdated. Version 0.11.7 was released Wednesday January 06, 2021.


Imputing missing values in feature set


Optimization Progress:   0%|          | 0/6 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -inf


RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly. If you enabled PyTorch estimators, please check the data requirements in the online documentation: https://epistasislab.github.io/tpot/using/

### Create a more detailed pipeline

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=42)

In [ ]:
# define search
pipeline_optimizer = TPOTClassifier(generations=3, # 5, # default 100
                      population_size=3, # 50,  # default 100
                      verbosity=2, 
                      random_state=42, 
                      scoring='roc_auc',
                      cv=cv,
                      warm_start=True,
                      periodic_checkpoint_folder='/tpot_checkpoints',
                      early_stop=3)

# perform search
pipeline_optimizer.fit(X_train, y_train)


In [ ]:
print(pipeline_optimizer.score(X_test, y_test.values))

# export the best model
pipeline_optimizer.export('tpot_digits_pipeline.py')